# Installs

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:
#!pip install --quiet cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl
!pip install --quiet "torch" "pytorch-lightning" "opencv-python==4.5.2.52" "scikit-learn==0.24.2" "torchmetrics" "torchvision==0.11.3" "pyod"
#!pip install --quiet "torch" "torchtext==0.9.0" "pytorch-lightning==1.3.3" "opencv-python==4.5.2.52" "scikit-learn==0.24.2" "torchmetrics>=0.3" "torchvision==0.11.3"
!pip install faiss-gpu==1.7.1.post3
!sudo apt-get install libopenblas-dev
!sudo apt-get install libomp-dev


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
#!git clone  https://github.com/hcw-00/PatchCore_anomaly_detection.git
#%cd PatchCore_anomaly_detection
#!ls -l

#Imports

In [ ]:
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
import matplotlib.pyplot as plt

import argparse
import torch
import torch.nn.functional as F
from torch import nn
import pytorch_lightning as pl
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import cv2
import numpy as np
import os
import glob
import shutil
from PIL import Image
from sklearn.metrics import roc_auc_score
from torch import nn

from tqdm.notebook import tqdm

from sklearn.metrics import confusion_matrix
import pickle
from sklearn.random_projection import SparseRandomProjection
from sklearn.neighbors import NearestNeighbors
from scipy.ndimage import gaussian_filter
import faiss

import tensorflow as tf

from tempfile import mkdtemp
import os.path as path
import numpy as np
import gc

print('Testing gpu availability: ', end='')
print(tf.test.gpu_device_name())



# DATASET

In [ ]:
class MVTecDataset(Dataset):
    def __init__(self, root, category, transform, gt_transform, phase, load_complete_data=False):
        if phase=='train':
          if load_complete_data:
            all_categories = [i for i in os.listdir(root) if os.path.isdir(os.path.join(root, i))]
            self.img_paths_root = [os.path.join(root, i_category, 'train') for i_category in all_categories]
          elif '_' in category:
            self.img_paths_root = [os.path.join(root, i_category, 'train') for i_category in category.split('_')]
          else:
            self.img_paths_root = [os.path.join(root, category, 'train')]
        
        else:
          if load_complete_data:
            all_categories = [i for i in os.listdir(root) if os.path.isdir(os.path.join(root, i))]
            self.img_paths_root = [os.path.join(root, i_category, 'test') for i_category in all_categories]
            self.gt_paths_root = [os.path.join(root, i_category, 'ground_truth') for i_category in all_categories]
          elif '_' in category:
            self.img_paths_root = [os.path.join(root, i_category, 'test') for i_category in category.split('_')]
            self.gt_paths_root = [os.path.join(root, i_category,'ground_truth') for i_category in category.split('_')]
          else:
            self.img_paths_root = [os.path.join(root, category, 'test')]
            self.gt_paths_root = [os.path.join(root, category,'ground_truth')]

        self.transform = transform
        self.gt_transform = gt_transform
        # load dataset
        self.img_paths, self.gt_paths, self.labels, self.types = self.load_dataset() # self.labels => good : 0, anomaly : 1

    def load_dataset(self):

        img_tot_paths = []
        gt_tot_paths = []
        tot_labels = []
        tot_types = []

        for ix, img_path in enumerate(self.img_paths_root):
          defect_types = os.listdir(img_path)

          for defect_type in defect_types:
              if defect_type == 'good':
                  img_paths = glob.glob(os.path.join(img_path, defect_type) + "/*.png")
                  img_tot_paths.extend(img_paths)
                  gt_tot_paths.extend([0]*len(img_paths))
                  tot_labels.extend([0]*len(img_paths))
                  #tot_types.extend(['good']*len(img_paths))
                  tot_types.extend([i.replace('../drive/MyDrive/data/mvtec_anomaly_detection/', '') for i in img_paths])
              else:
                  gt_path = self.gt_paths_root[ix]
                  img_paths = glob.glob(os.path.join(img_path, defect_type) + "/*.png")
                  gt_paths = glob.glob(os.path.join(gt_path, defect_type) + "/*.png")
                  img_paths.sort()
                  gt_paths.sort()
                  img_tot_paths.extend(img_paths)
                  gt_tot_paths.extend(gt_paths)
                  tot_labels.extend([1]*len(img_paths))
                  #tot_types.extend([defect_type]*len(img_paths))
                  tot_types.extend([i.replace('../drive/MyDrive/data/mvtec_anomaly_detection/', '') for i in img_paths])

        assert len(img_tot_paths) == len(gt_tot_paths), "Something wrong with test and ground truth pair!"
        return img_tot_paths, gt_tot_paths, tot_labels, tot_types

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path, gt, label, img_type = self.img_paths[idx], self.gt_paths[idx], self.labels[idx], self.types[idx]
        img = Image.open(img_path).convert('RGB')
        img = self.transform(img)
        if gt == 0:
            gt = torch.zeros([1, img.size()[-2], img.size()[-2]])
        else:
            gt = Image.open(gt)
            gt = self.gt_transform(gt)
        
        assert img.size()[1:] == gt.size()[1:], "image.size != gt.size !!!"

        return img, gt, label, os.path.basename(img_path[:-4]), img_type

# Filemanagement

In [ ]:
def copy_files(src, dst, ignores=[]):
    src_files = os.listdir(src)
    for file_name in src_files:
        ignore_check = [True for i in ignores if i in file_name]
        if ignore_check:
            continue
        full_file_name = os.path.join(src, file_name)
        if os.path.isfile(full_file_name):
            shutil.copy(full_file_name, os.path.join(dst,file_name))
        if os.path.isdir(full_file_name):
            os.makedirs(os.path.join(dst, file_name), exist_ok=True)
            copy_files(full_file_name, os.path.join(dst, file_name), ignores)

def prep_dirs(root):
    # make embeddings dir
    # embeddings_path = os.path.join(root, 'embeddings')
    embeddings_path = os.path.join('./', 'embeddings', args.category)
    os.makedirs(embeddings_path, exist_ok=True)
    # make sample dir
    sample_path = os.path.join(root, 'sample')
    os.makedirs(sample_path, exist_ok=True)
    # make source code record dir & copy
    source_code_save_path = os.path.join(root, 'src')
    os.makedirs(source_code_save_path, exist_ok=True)
    copy_files('./', source_code_save_path, ['.git','.vscode','__pycache__','logs','README','samples','LICENSE']) # copy source code
    return embeddings_path, sample_path, source_code_save_path


# Evaluation

In [ ]:
def cal_confusion_matrix(y_true, y_pred_no_thresh, thresh, img_path_list):
    pred_thresh = []
    false_n = []
    false_p = []
    for i in range(len(y_pred_no_thresh)):
        if y_pred_no_thresh[i] > thresh:
            pred_thresh.append(1)
            if y_true[i] == 0:
                false_p.append(img_path_list[i])
        else:
            pred_thresh.append(0)
            if y_true[i] == 1:
                false_n.append(img_path_list[i])

    cm = confusion_matrix(y_true, pred_thresh)
    print(cm)
    print('false positive')
    print(false_p)
    print('false negative')
    print(false_n)
    return cm, false_p, false_n
    

# Sampling

In [ ]:
# Copyright 2017 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Returns points that minimizes the maximum distance of any point to a center.
Implements the k-Center-Greedy method in
Ozan Sener and Silvio Savarese.  A Geometric Approach to Active Learning for
Convolutional Neural Networks. https://arxiv.org/abs/1708.00489 2017
Distance metric defaults to l2 distance.  Features used to calculate distance
are either raw features or if a model has transform method then uses the output
of model.transform(X).
Can be extended to a robust k centers algorithm that ignores a certain number of
outlier datapoints.  Resulting centers are solution to multiple integer program.
"""

from sklearn.metrics import pairwise_distances
import abc
import numpy as np

class SamplingMethod(object):
  __metaclass__ = abc.ABCMeta

  @abc.abstractmethod
  def __init__(self, X, y, seed, **kwargs):
    self.X = X
    self.y = y
    self.seed = seed

  def flatten_X(self):
    shape = self.X.shape
    flat_X = self.X
    if len(shape) > 2:
      flat_X = np.reshape(self.X, (shape[0],np.product(shape[1:])))
    return flat_X


  @abc.abstractmethod
  def select_batch_(self):
    return

  def select_batch(self, **kwargs):
    return self.select_batch_(**kwargs)

  def to_dict(self):
    return None

class kCenterGreedy(SamplingMethod):

  def __init__(self, X, y, seed, metric='euclidean'):
    self.X = X
    self.y = y
    self.flat_X = self.flatten_X()
    self.name = 'kcenter'
    self.features = self.flat_X
    self.metric = metric
    self.min_distances = None
    self.n_obs = self.X.shape[0]
    self.already_selected = []

  def update_distances(self, cluster_centers, only_new=True, reset_dist=False):
    """Update min distances given cluster centers.
    Args:
      cluster_centers: indices of cluster centers
      only_new: only calculate distance for newly selected points and update
        min_distances.
      rest_dist: whether to reset min_distances.
    """

    if reset_dist:
      self.min_distances = None
    if only_new:
      cluster_centers = [d for d in cluster_centers
                         if d not in self.already_selected]
    if cluster_centers:
      # Update min_distances for all examples given new cluster center.
      x = self.features[cluster_centers]
      dist = pairwise_distances(self.features, x, metric=self.metric)

      if self.min_distances is None:
        self.min_distances = np.min(dist, axis=1).reshape(-1,1)
      else:
        self.min_distances = np.minimum(self.min_distances, dist)

  def select_batch_(self, model, already_selected, N, **kwargs):
    """
    Diversity promoting active learning method that greedily forms a batch
    to minimize the maximum distance to a cluster center among all unlabeled
    datapoints.
    Args:
      model: model with scikit-like API with decision_function implemented
      already_selected: index of datapoints already selected
      N: batch size
    Returns:
      indices of points selected to minimize distance to cluster centers
    """

    try:
      # Assumes that the transform function takes in original data and not
      # flattened data.
      print('Getting transformed features...')
      if model:
        self.features = model.transform(self.X)
      else:
        self.features = self.X
      
      print('Calculating distances...')
      self.update_distances(already_selected, only_new=False, reset_dist=True)
    except:
      print('Using flat_X as features.')
      self.update_distances(already_selected, only_new=True, reset_dist=False)

    new_batch = []

    for _ in tqdm(range(N)):
      if self.already_selected is None:
        # Initialize centers with a randomly selected datapoint
        ind = np.random.choice(np.arange(self.n_obs))
      else:
        ind = np.argmax(self.min_distances)
      # New examples should not be in already selected since those points
      # should have min_distance of zero to a cluster center.
      assert ind not in already_selected

      self.update_distances([ind], only_new=True, reset_dist=False)
      new_batch.append(ind)

    self.already_selected = already_selected
    return new_batch

In [ ]:
def embedding_concat(x, y):
    # from https://github.com/xiahaifeng1995/PaDiM-Anomaly-Detection-Localization-master
    B, C1, H1, W1 = x.size()
    _, C2, H2, W2 = y.size()
    s = int(H1 / H2)
    x = F.unfold(x, kernel_size=s, dilation=1, stride=s)
    x = x.view(B, C1, -1, H2, W2)
    z = torch.zeros(B, C1 + C2, x.size(2), H2, W2)
    for i in range(x.size(2)):
        z[:, :, i, :, :] = torch.cat((x[:, :, i, :, :], y), 1)
    z = z.view(B, -1, H2 * W2)
    z = F.fold(z, kernel_size=s, output_size=(H1, W1), stride=s)

    return z

def reshape_embedding(embedding):
    embedding_list = []
    for k in range(embedding.shape[0]):
        for i in range(embedding.shape[2]):
            for j in range(embedding.shape[3]):
                embedding_list.append(embedding[k, :, i, j])
    return embedding_list


# PatchCore Multiclass

In [ ]:
class STPM(pl.LightningModule):
    def __init__(self, hparams):
        super(STPM, self).__init__()
        #self.save_hyperparameters(hparams)

        self.init_features()
        def hook_t(module, input, output):
            self.features.append(output)

        torch.hub._validate_not_a_forked_repo=lambda a,b,c: True # bug workaround to load resnet correctly

        self.model = torch.hub.load('pytorch/vision:v0.9.0', 'wide_resnet50_2', pretrained=True)
        self.model.layer2[-1].register_forward_hook(hook_t)
        self.model.layer3[-1].register_forward_hook(hook_t)

        for param in self.model.parameters():
            param.requires_grad = False

        self.criterion = torch.nn.MSELoss(reduction='sum')

        self.init_results_list()

        #imagenet
        mean_train = [0.485, 0.456, 0.406]
        std_train = [0.229, 0.224, 0.225]

        self.data_transforms = transforms.Compose([
                        transforms.Resize((args.load_size, args.load_size), Image.ANTIALIAS),
                        transforms.ToTensor(),
                        transforms.CenterCrop(args.input_size),
                        transforms.Normalize(mean=mean_train,
                                            std=std_train)])
        self.gt_transforms = transforms.Compose([
                        transforms.Resize((args.load_size, args.load_size)),
                        transforms.ToTensor(),
                        transforms.CenterCrop(args.input_size)])

        self.inv_normalize = transforms.Normalize(mean=[-0.485/0.229, -0.456/0.224, -0.406/0.255], std=[1/0.229, 1/0.224, 1/0.255])

    def init_results_list(self):
        self.gt_list_px_lvl = []
        self.pred_list_px_lvl = []
        self.gt_list_img_lvl = []
        self.pred_list_img_lvl = []
        self.img_path_list = []        

    def init_features(self):
        self.features = []

    def forward(self, x_t):
        self.init_features()
        _ = self.model(x_t)
        return self.features

    def train_dataloader(self):
        image_datasets = MVTecDataset(root=os.path.join(args.dataset_path),
                                      category=args.category,
                                      transform=self.data_transforms,
                                      gt_transform=self.gt_transforms,
                                      phase='train',
                                      load_complete_data=args.load_complete_data)
        train_loader = DataLoader(image_datasets,
                                  batch_size=args.batch_size,
                                  shuffle=True,
                                  num_workers=4,
                                  pin_memory=True)
        return train_loader

    def test_dataloader(self):
        test_datasets = MVTecDataset(root=os.path.join(args.dataset_path),
                                     category=args.category,
                                     transform=self.data_transforms,
                                     gt_transform=self.gt_transforms,
                                     phase='test',
                                     load_complete_data=args.load_complete_data)
        test_loader = DataLoader(test_datasets,
                                 batch_size=1,
                                 shuffle=False,
                                 num_workers=4, 
                                 pin_memory=True) # only work on batch_size=1, now.
        return test_loader

    def configure_optimizers(self):
        return None

    def on_train_start(self):
        self.model.eval() # to stop running_var move (maybe not critical)
        _, self.sample_path, self.source_code_save_path = prep_dirs(self.logger.log_dir)

        self.embedding_dir_path = f'../drive/MyDrive/data/wideresnet_embedding/{args.category}'
        if not os.path.isdir(self.embedding_dir_path):
          os.mkdir(self.embedding_dir_path)
        self.embedding_list = []

    def on_test_start(self):
      
        self.embedding_dir_path = f'../drive/MyDrive/data/wideresnet_embedding/{args.category}'
        self.index = faiss.read_index(os.path.join(self.embedding_dir_path,'index.faiss'))
        if torch.cuda.is_available():
            res = faiss.StandardGpuResources()
            self.index = faiss.index_cpu_to_gpu(res, 0 ,self.index)
        self.init_results_list()
        #self.embedding_dir_path, self.sample_path, self.source_code_save_path = prep_dirs(self.logger.log_dir)

    def training_step(self, batch, batch_idx): # save locally aware patch features
        x, _, _, file_name, _ = batch
        features = self(x)
        embeddings = []
        for feature in features:
            m = torch.nn.AvgPool2d(3, 1, 1)
            embeddings.append(m(feature))
        embedding = embedding_concat(embeddings[0], embeddings[1])
        self.embedding_list.extend(reshape_embedding(np.array(embedding)))

    def training_epoch_end(self, outputs): 
        total_embeddings = np.array(self.embedding_list)
        # Random projection
        self.randomprojector = SparseRandomProjection(n_components='auto', eps=0.9) # 'auto' => Johnson-Lindenstrauss lemma
        self.randomprojector.fit(total_embeddings)
        # Coreset Subsampling
        selector = kCenterGreedy(total_embeddings,0,0)
        selected_idx = selector.select_batch(model=self.randomprojector, already_selected=[], N=int(total_embeddings.shape[0]*args.coreset_sampling_ratio))
        self.embedding_coreset = total_embeddings[selected_idx]
        
        # print('initial embedding size : ', total_embeddings.shape)
        print('final embedding size : ', self.embedding_coreset.shape)
        #faiss
        self.index = faiss.IndexFlatL2(self.embedding_coreset.shape[1])
        self.index.add(self.embedding_coreset) 
        faiss.write_index(self.index,  os.path.join(self.embedding_dir_path,'index.faiss'))
        with open(os.path.join(self.embedding_dir_path,'embedding.pickle'), 'wb') as f:
          pickle.dump(self.embedding_coreset, f) 

    def test_step(self, batch, batch_idx): # Nearest Neighbour Search
        x, gt, label, file_name, x_type = batch
        # extract embedding
        features = self(x)
        embeddings = []
        for feature in features:
            m = torch.nn.AvgPool2d(3, 1, 1)
            embeddings.append(m(feature))
        embedding_ = embedding_concat(embeddings[0], embeddings[1])
        embedding_test = np.array(reshape_embedding(np.array(embedding_)))
        score_patches, _ = self.index.search(embedding_test , k=args.n_neighbors)
        N_b = score_patches[np.argmax(score_patches[:,0])]
        w = (1 - (np.max(np.exp(N_b))/np.sum(np.exp(N_b))))
        score = w*max(score_patches[:,0]) # Image-level score
        gt_np = gt.cpu().numpy()[0,0].astype(int)
        self.gt_list_px_lvl.extend(gt_np.ravel())
        self.gt_list_img_lvl.append(label.cpu().numpy()[0])
        self.pred_list_img_lvl.append(score)
        self.img_path_list.extend(x_type)

    def test_epoch_end(self, outputs):
        print("Total image-level auc-roc score :")
        img_auc = roc_auc_score(self.gt_list_img_lvl, self.pred_list_img_lvl)
        print(img_auc)
        print('test_epoch_end')
        
        safe_path = f'../drive/MyDrive/data/wideresnet_multi/{args.category}.pickle'
        with open(safe_path, 'wb') as f:
            pickle.dump({'gt_list_img_lvl': self.gt_list_img_lvl,
                         'pred_list_img_lvl': self.pred_list_img_lvl,
                         'img_path_list': self.img_path_list}, f)

# RUN

In [ ]:
import os
import shutil
if not 'run_from_here' in os.getcwd():
  !mkdir run_from_here
  %cd run_from_here
else:
  print( os.getcwd())
def _empty_cache():
  torch.cuda.empty_cache()
  gc.collect()
  gc.collect()

In [ ]:
  def get_args(name='carpet', layers=None):
    parser = argparse.ArgumentParser(description='ANOMALYDETECTION')
    parser.add_argument('--phase', choices=['train','test', 'train_test'], default='test')
    parser.add_argument('--dataset_path', default='../drive/MyDrive/data/mvtec_anomaly_detection')
    parser.add_argument('--category', default=None)
    parser.add_argument('--num_epochs', default=1)
    parser.add_argument('--batch_size', default=32)
    parser.add_argument('--load_size', default=256) # 256
    parser.add_argument('--input_size', default=224)
    parser.add_argument('--contamination', default=0.01)
    parser.add_argument('--pytorch_version', default='NVIDIA/DeepLearningExamples:torchhub')#'pytorch/vision:v0.10.0')
    parser.add_argument('--model_name', default='nvidia_efficientnet_b4')#'wide_resnet50_2')
    parser.add_argument('--coreset_sampling_ratio', default=0.05)
    parser.add_argument('--project_root_path', default='./')
    parser.add_argument('--result_path', default='../drive/MyDrive/data/mvtec_results/')
    parser.add_argument('--save_src_code', default=False)
    parser.add_argument('--save_anomaly_map', default=False)
    parser.add_argument('--n_neighbors', type=int, default=9)
    parser.add_argument('--load_complete_data', default=False)
    parser.add_argument('--nearest_neighbors', type=int, default=9)
    parser.add_argument('--distance_batch_size', default=1000)
    
    args = parser.parse_args(['--category', name])

    return args


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

_to_test = ['bottle', 'cable', 'capsule', 'carpet', 'grid', 
            'hazelnut', 'leather', 'metalnut',
            'pill', 'screw', 
            'tile', 'toothbrush', 'transistor', 'wood', 'zipper']

totest_joined = []
for i in _to_test:
   for j in _to_test:
     joined = '_'.join(sorted([i, j]))
     if joined not in totest_joined and i != j:
      totest_joined.append(joined)

for name in totest_joined:

    args = get_args(name)
    print(args.category)
    safe_path = f'../drive/MyDrive/data/wideresnet_multi/{args.category}.pickle'
    if os.path.isfile(safe_path):
      print('already finished')
      continue
      
    trainer = Trainer.from_argparse_args(args,
                                         default_root_dir=os.path.join(args.project_root_path, args.category),
                                         max_epochs=args.num_epochs,
                                         gpus=1,
                                         log_every_n_steps=1)
    model = STPM(hparams=args)
    if args.phase == 'train':
        trainer.fit(model)
        _empty_cache()
    elif args.phase == 'test':
        trainer.test(model)
        _empty_cache()
    elif args.phase == 'train_test':
        trainer.fit(model)
        _empty_cache()
        trainer.test(model)
    _empty_cache()

# Multiclass vs Singleclass Evaluation

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
from sklearn.metrics import roc_auc_score
import os
_to_test = ['bottle', 'cable', 'capsule', 'carpet', 'grid', 
            'hazelnut', 'leather', 'metalnut', 'pill', 'screw', 
            'tile', 'toothbrush', 'transistor', 'wood', 'zipper']


totest_joined = []
for i in _to_test:
   for j in _to_test:
     joined = '_'.join(sorted([i, j]))
     if joined not in totest_joined and i != j:
      totest_joined.append(joined)

In [ ]:
 # Load single results
base_path_single = f'drive/MyDrive/data/wideresnet_evaluation/results_patchcore.pickle'
with open(base_path_single, "rb") as input_file:
  results_single = pickle.load(input_file)

results_single_flat = {i: results_single[i][9] for i in results_single}
results_single_flat["metalnut"] = results_single_flat.pop("metal_nut")
results_single_flat

In [ ]:
results_multi = {}
for category in totest_joined:
  safe_path = f'drive/MyDrive/data/wideresnet_multi/{category}.pickle'
  if os.path.isfile(safe_path):
    with open(safe_path, "rb") as input_file:
      e = pickle.load(input_file)
      results_multi[category] = e

In [ ]:
results_multi_imgauc = {}
for k, v in results_multi.items():
  img_path_list = v['img_path_list']
  c1, c2 = k.split('_')
  gt1, gt2, pred1, pred2 = [], [], [], []
  
  for ix, val in enumerate(img_path_list):
    if c1 in val: 
      gt1.append(v['gt_list_img_lvl'][ix])
      pred1.append(v['pred_list_img_lvl'][ix])
    elif c2 in val:
      gt2.append(v['gt_list_img_lvl'][ix])
      pred2.append(v['pred_list_img_lvl'][ix])
    else:
      print('something is wrong')

  results_multi_imgauc[k] = {c1: roc_auc_score(gt1, pred1),
                             c2: roc_auc_score(gt2, pred2)}

results_multi_imgauc

In [ ]:
data = []
mean_single = 0
for cat in results_single_flat:
  row = {'Kategorie 1': cat + ' - ' + str(results_single_flat[cat])[:5]}
  mean_single += results_single_flat[cat]
  for cat2 in results_single_flat:
    if cat == cat2:
      row[cat2] = None
    else:
      row[cat2] = results_multi_imgauc['_'.join(sorted([cat, cat2]))][cat] - results_single_flat[cat]
  data.append(row)

mean_single /= len(results_single_flat)
df = pd.DataFrame(data)
df.set_index('Kategorie 1', inplace=True)
df.loc['mean - ' + str(mean_single)[:5]] = df.mean()
df['mean'] = df.mean(axis=1)

In [ ]:
print(df.min().min())
print(df.max().max())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(16,8))
ax = sns.heatmap(df, annot=True, cmap="RdYlGn", fmt=".4f", vmin=-0.0085, vmax=.0085, center=0, linewidths=.2)
plt.title('Mehrklassen vs. Einklassen Anomaliedetektion', fontdict={'fontsize': 24}, loc='left')
plt.xticks(rotation=30)
plt.tight_layout()

plt.savefig(f'drive/MyDrive/data/wideresnet_multi/lift_multi_single.png', dpi=400)